In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.distributions.empirical_distribution import ECDF

column_names = ['TPSA(Tot)', 'SAacc', 'H-050', 'MLOGP', 'RDCHI', 'GATS1p', 'nN', 'C-040', 'LC50']
df = pd.read_csv('qsar_aquatic_toxicity.csv', sep = ';', names = column_names)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

def split_data(X, y, train_size=0.5, unlabeled_size=0.2):
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.3)
    train_size_relative = train_size / (train_size + unlabeled_size)
    X_train, X_unlabeled, y_train, _ = train_test_split(X_train_full, y_train_full, train_size=train_size_relative)
    return (X_train, y_train), (X_unlabeled, _), (X_test, y_test)

def calculate_error_rejection_rate(model, model_params, epsilon_values, X, y, num_iterations=30):
    results = {eps: [] for eps in epsilon_values}

    for eps in epsilon_values:
        for _ in range(num_iterations):
            (X_train, y_train), (X_unlabeled, _), (X_test, y_test) = split_data(X, y)            
           
            reg = model(**model_params)
            reg.fit(X_train, y_train)
            y_pred_train = reg.predict(X_train)
            residuals = (y_train - y_pred_train) ** 2
            
            reg_residuals = model(**model_params)
            reg_residuals.fit(X_train, residuals)
            
            ecdf = ECDF(reg_residuals.predict(X_unlabeled))
            test_residuals = reg_residuals.predict(X_test)
            scores = ecdf(test_residuals)
            accepted_indices = np.where(scores <= 1 - eps)[0]
            
            if len(accepted_indices) > 0:
                accepted_predictions = reg.predict(X_test.iloc[accepted_indices])
                error_rate = np.mean((y_test.iloc[accepted_indices] - accepted_predictions) ** 2)
            else:
                error_rate = np.nan
            rejection_rate = 1 - len(accepted_indices) / len(y_test)
            results[eps].append((error_rate, rejection_rate))

    return results

models = {
    "KNN": (KNeighborsRegressor, {'n_neighbors': 10}),
    "Random Forest": (RandomForestRegressor, {'n_estimators': 100}),
    "SVM": (SVR, {'kernel': 'linear', 'C': 1})
}

epsilon_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
all_results = {}

for model_name, (model, params) in models.items():
    all_results[model_name] = calculate_error_rejection_rate(model, params, epsilon_values, X, y)

for model_name, results in all_results.items():
    print(f"Model: {model_name}")
    for eps in epsilon_values:
        res = np.array(results[eps])
        mean_error = np.nanmean(res[:, 0])
        std_error = np.nanstd(res[:, 0])
        mean_rejection = np.mean(res[:, 1])
        std_rejection = np.std(res[:, 1])
        print(f"Epsilon: {eps}, Mean Error: {mean_error}, Std Error: {std_error}, Mean Rejection: {mean_rejection}, Std Rejection: {std_rejection}")
    print("\n")

for model_name, results in all_results.items():
    mean_errors = [np.nanmean(np.array(results[eps])[:, 0]) for eps in epsilon_values]
    plt.plot(epsilon_values, mean_errors, label=model_name, marker='o')

plt.xlabel('1-$\epsilon$')
plt.ylabel('$\hat{E_{rr}}$')
plt.title('Aquatic Dataset')
plt.legend()
plt.show()